<a href="https://colab.research.google.com/github/ria-heikkila/GAN_mnist/blob/master/GAN_mnist.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

![代替テキスト](https://www.imagazine.co.jp/wp-content/uploads/2018/09/is20tech001zu004-1.jpg)

生成・学習に多層NNを使用</br>
＊CNNを使うと「DCGAN」となる

In [0]:
%matplotlib inline

import pickle as pkl
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt


In [0]:
#学習データと検証データをアップロード
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('MNIST_data')

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


データの入れ物は「プレースホルダー」→　データの容器</br>
実行時にfeed_dictという引数でプレースホルダーにデータを渡す

---



**GANの構築**

１．モデルの定義（登場する変数を定義する（D、G））</br>
２．損失関数（Loss）を定義する</br>
３．最適化手法を定義する</br>




In [0]:
#真データとノイズの次元（サイズ）を指定
def model_inputs(read_dim, z_dim):
  #真データ
  inputs_real = tf.placeholder(tf.float32, (None, real_dim), name = 'input_real') #データの個数はまだわからないので、None（未定）で指定しておく
  #ノイズ
  inputs_z = tf.placeholder(tf.float32, (None, z_dim), name ='input_z')

  return inputs_real, inputs_z


In [0]:
#ジェネレーターの定義
#reuseオプション：関数内の変数値の保持
 def generator(z, out_dim, n_units=128, reuse=False, alpha = 0.01):
   with tf.variable_scope('generator', reuse=reuse): #reuseオプションの使用場面を指定
     #h1：1つ目の隠れ層のノードの値
     h1 = tf.layers.dense(z, n_units, activation=None) #Leaky ReLuはtensorflowに入ってないので、Noneにし、下記でLeaky ReLuを自分で定義する
     #Leaky ReLu
     h1 = tf.maximum(alpha * h1, h1)

     #logits = score 
     logits = tf.layers.dense(h1, out_dim, activation=None)
     #tanh(ハイパボリックタンジェント：-1～1の値を出力する)
     out = tf.tanh(logits)

     return 

tanhとsigmoidの違い

![tanhとsigmoidの違い](https://sebastianraschka.com/images/faq/tanh-sigmoid-relationship/tanh-sigmoid.png)

In [0]:
#ディスクリミネーターの定義
def discriminator(x, n_units=128, reuse=False, alpha=0.01):
  with tf.variable_scope('discriminator', reuse=reuse):
    h1 = tf.layers.dense(x, n_units, activation = None)
    h1 = tf.maximum(alpha*h1,h1)

    logits = tf.layers.dense(h1, 1, activation = None)
    out = tf.sigmoid(logits)

    return out, logits

In [0]:
#ハイパーパラメータの初期化

input_size = 784
z_size = 100
g_hidden_size = 128
d_hidden_size = 128
alpha = 0.01 #Leaky ReLuの負領域
smooth = 0.1 #Dの学習を円滑にする調整

In [0]:
#計算グラフの定義
 